In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(r'delivery_orders_march.csv')
df.head(3)

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."


In [59]:
def roadmap(start, dest):
    city = {"Metro Manila": 0, "Luzon": 1, "Visayas":2, "Mindanao":3}
    sla = [[3,5,7,7], [5,5,7,7], [7,7,7,7], [7,7,7,7]]
    return sla[city[start]][city[dest]]
# roadmap("Metro Manila","Luzon")

In [74]:
places = ["Metro Manila", "Luzon", "Visayas", "Mindanao"]
def check_buyerplaces(row):
    for b in places:
        if b in row['buyeraddress'].lower():
            buy = b
            return buy
def check_sellerplaces(row):
    for s in places:
        if s in row['selleraddress'].lower():
            sell = s
            return sell

In [75]:
df['buyer'] = df.apply(lambda x: check_buyerplaces(x), axis = 1)
df['seller'] = df.apply(lambda x: check_sellerplaces(x), axis = 1)

In [76]:
df.head(3)

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,day_difference,buyer,seller
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,normal,None,None
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",Late,None,None
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",normal,None,None


In [1]:
# df['supposed_days'] = df.apply(lambda x: roadmap(x['seller'], x['buyer']), axis = 1)

In [56]:
from datetime import timedelta
#generate datetime obj
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)
# input datetime obj: start, end                                   
def workingDay(start_date, end_date):
    holiday_count = 0
    cnt = 0
    date_gen = daterange(start_date, end_date)
    next(date_gen) #skip the pickup date
    for dateNow in date_gen:
        print("now", dateNow.strftime("%Y-%m-%d"), dateNow.weekday())
        if dateNow.weekday() == 6:
            holiday_count += 1
        elif (dateNow - datetime.strptime('2020-03-08','%Y-%m-%d')).days == 0:
            holiday_count += 1
        elif (dateNow - datetime.strptime('2020-03-25','%Y-%m-%d')).days == 0:
            holiday_count += 1
        elif (dateNow - datetime.strptime('2020-03-30','%Y-%m-%d')).days == 0:
            holiday_count += 1
        elif (dateNow - datetime.strptime('2020-03-31','%Y-%m-%d')).days == 0:
            holiday_count += 1
        cnt += 1
    return cnt - holiday_count

def convert_time(row):
    first = datetime.datetime.strptime(datetime.datetime.fromtimestamp(row['1st_deliver_attempt']))
    if np.isnan(row['2nd_deliver_attempt']):
        return 0
    else:
        second = datetime.datetime.strptime(datetime.datetime.fromtimestamp(row['2nd_deliver_attempt']))#.strftime('%Y-%m-%d'), '%Y-%m-%d')
        if workingDay(first, second) > 3:
            return 1
        else:
            return str(workingDay(first, second)) + " days"

In [2]:
# df['day_difference'] = df.apply(lambda x: convert_time(x), axis = 1)
# df['day_difference']

In [9]:
sla = pd.read_excel(r'SLA_matrix.xlsx')
print(sla.shape)
sla.head(3)

(9, 6)


,1st Attempt SLA\n(Working Days),Unnamed: 1,Destination (Buyer),Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,Metro Manila,Luzon,Visayas,Mindanao
1,Origin\n(Seller),Metro Manila,3 working days,5 working days,7 working days,7 working days
2,NaN,Luzon,5 working days,5 working days,7 working days,7 working days


In [5]:
df.shape

(3176313, 6)

In [6]:
second = df['2nd_deliver_attempt'].isnull().sum()
second/df.shape[0]

0.5727744715335044

In [44]:
baseline_submit = pd.DataFrame()
baseline_submit['orderid'] = df['orderid'].unique()
baseline_submit['is_late'] = 1
baseline_submit.shape
baseline_submit.to_csv(r'baseline_submission.csv', index = False)

In [45]:
base = pd.read_csv(r'baseline_submission.csv')
base.head(3)

,orderid,is_late
0,2215676524,1
1,2219624609,1
2,2220979489,1
